# Bilan hydrique à partir d'observations Météo-France quotidiennes qualifiées

## Reconstruction des données météorologiques pour une station de référence

### Définition des paramètres

In [1]:
# Récupération par lecture ou par téléchargement
LIRE_DONNEE = False
LIRE_LISTE_STATIONS = False
LIRE_DONNEE_REF = False

# Définition de la station de référence
REF_STATION_NAME = # "Mon site de référence"
REF_STATION_LATLON = # [50., 0.]
# Altitude (m)
REF_STATION_ALTITUDE = # 10

# Départements où chercher des stations
ID_DEPARTEMENTS = # [1, ...]

# Plus proches voisins
NN_NOMBRE = # None
NN_RAYON_KM = # 10

# Période météo (une ou plusieurs années complètes consécutives)
DATE_DEB_PERIODE = # '2024-01-01T00:00:00Z'
DATE_FIN_PERIODE = # '2024-12-31T00:00:00Z'

# Identification de l'API Météo-France
APPLICATION_ID = # "AbC1..."

### Lecture de la liste des stations

In [2]:
import meteofrance

# Météo-France API
METEOFRANCE_API = 'DPClim'

# Initialisation d'un client pour accéder à l'API Météo-France
client = meteofrance.Client(METEOFRANCE_API, application_id=APPLICATION_ID)

In [3]:
import pandas as pd

# Fréquence des données climatiques
METEOFRANCE_FREQUENCE = 'quotidienne'

l_listes = []
for id_dep in ID_DEPARTEMENTS:
    filepath_liste_stations = meteofrance.get_filepath_liste_stations(
        client, frequence=METEOFRANCE_FREQUENCE, id_departement=id_dep)
    
    if LIRE_LISTE_STATIONS:
        # Lecture de la liste des stations par département
        df_liste_stations_dep = pd.read_csv(
            filepath_liste_stations, index_col=client.id_station_label)
    else:
        # Demande de la liste des stations pour le département
        section = meteofrance.SECTION_LISTE_STATIONS
        params = {'id-departement': id_dep}
        response = meteofrance.demande(
            client, section, params=params, frequence=METEOFRANCE_FREQUENCE)
        df_liste_stations_dep = meteofrance.response_text_to_frame(
            client, response, index_col=client.id_station_label)

        # Sauvegarde de la liste des stations par département
        df_liste_stations_dep.to_csv(filepath_liste_stations)

    # Liste pour compilation
    l_listes.append(df_liste_stations_dep)

# Compilation
df_liste_stations_brute = pd.concat(l_listes, axis='index')

# Garder les stations valides seulement
df_liste_stations = meteofrance.filtrer_stations_valides(client, df_liste_stations_brute)

### Sélection des plus proches voisins de la station de référence

In [4]:
import geo

df_liste_stations_nn = geo.selection_stations_plus_proches(
    df_liste_stations, REF_STATION_LATLON, client.latlon_labels,
    nombre=NN_NOMBRE, rayon_km=NN_RAYON_KM)

df_liste_stations_nn

,nom,typePoste,lon,lat,alt,distance
id,,,,,,
35044001,BROUALAN,2,-1.640833,48.485667,99,6
50410003,PONTORSON,1,-1.505167,48.585667,33,9
35224001,PLERGUER,2,-1.843667,48.524833,30,17


### Obtention des données météorologiques pour les stations voisines

In [5]:
import bilan

# Ségmentation de la période par année
idx_dates_deb = pd.date_range(
    start=DATE_DEB_PERIODE, end=DATE_FIN_PERIODE, freq='YS-JAN')
idx_dates_fin = pd.date_range(
    start=DATE_DEB_PERIODE, end=DATE_FIN_PERIODE, freq='YE-DEC')
list_dates_deb = [d.isoformat().replace("+00:00", "Z") for d in idx_dates_deb]
list_dates_fin = [d.isoformat().replace("+00:00", "Z") for d in idx_dates_fin]

df_meteo = pd.DataFrame(dtype=float)
for date_deb, date_fin in zip(list_dates_deb, list_dates_fin):
    filepath_donnee_an = meteofrance.get_filepath_donnee_periode(
        client, REF_STATION_NAME, df_liste_stations_nn, date_deb, date_fin,
        frequence=METEOFRANCE_FREQUENCE)
    
    if LIRE_DONNEE:
        # Lecture des données des stations pour la période
        df_meteo_an = pd.read_csv(
            filepath_donnee_an, parse_dates=[client.time_label],
            index_col=[client.id_station_donnee_label, client.time_label])
    else:
        # Demande des données des stations pour la période
        variables = [client.variables_labels[METEOFRANCE_FREQUENCE][k]
                     for k in bilan.VARIABLES_CALCUL_BILAN]
        df_meteo_an = meteofrance.compiler_telechargement_des_stations_periode(
            client, df_liste_stations_nn, date_deb, date_fin,
            frequence=METEOFRANCE_FREQUENCE)[variables]
    
        # Sauvegarde des données des stations pour la période par département
        df_meteo_an.to_csv(filepath_donnee_an)

    # Compilation des années
    df_meteo = pd.concat([df_meteo, df_meteo_an], axis='index')

### Interpolation des données météorologiques à la station de référence

Les variables sont également renommées en utilisant des noms communs à l'ensemble de ces notebooks quelque soit l'API utilisée.

In [6]:
filepath_donnee_ref = meteofrance.get_filepath_donnee_periode(
    client, REF_STATION_NAME, df_liste_stations_nn, DATE_DEB_PERIODE, DATE_FIN_PERIODE,
    frequence=METEOFRANCE_FREQUENCE, ref=True)

if LIRE_DONNEE_REF:
    # Lecture des données des stations pour la période
    df_meteo_ref = pd.read_csv(
        filepath_donnee_ref, parse_dates=[client.time_label],
        index_col=client.time_label)
else:
    # Demande des données des stations pour la période
    df_meteo_ref = geo.interpolation_inverse_distance_carre(
        df_meteo, df_liste_stations_nn['distance'])
    
    # Sauvegarde par département
    df_meteo_ref.to_csv(filepath_donnee_ref)

df_meteo_ref = meteofrance.renommer_variables(
    client, df_meteo_ref, METEOFRANCE_FREQUENCE)

### Conversion des unités

In [7]:
df_meteo_ref_si = meteofrance.convertir_unites(
    client, df_meteo_ref)

## Estimation du bilan hydrique

### Définition des paramètres

In [8]:
# RFU finale cible (mm)
RFU_CIBLE = None

# Fraction du sol occupé par des cailloux et graviers (entre 0 pour absence de cailloux et 1 pour totalité de cailloux)
FRACTION_CAILLOUX = 0.1

# Choix de la texture
TEXTURE = 'Terres limoneuses'

# Coefficient de conversion de la RU en RFU (entre 1/2 et 2/3)
RU_VERS_RFU = 2. / 3

# Fraction de la réserve utile du sol remplie d'eau (entre 0 pour une période sèche et 1 pour une période pluvieuse)
FRACTION_RU_REMPLIE = 1.

# Besoin d'irrigation minimal à partir du quel irriguer (mm)
SEUIL_IRRIGATION = 1.

# Conversion de hauteur (mm) vers durée d'irrigation (min)
HAUTEUR_VERS_DUREE_IRRIGATION = 10

### Calcul du bilan hydrique

In [9]:
# Choix de la culture
CULTURE = list(bilan.KC)[0]

# Choix du stade
STADE = list(bilan.KC[CULTURE])[0]

df_bilan = bilan.calcul_bilan(
    df_meteo_ref_si,
    TEXTURE, FRACTION_CAILLOUX,
    CULTURE, STADE,
    FRACTION_RU_REMPLIE, RU_VERS_RFU,
    seuil_irrigation=SEUIL_IRRIGATION,
    hauteur_vers_duree_irrigation=HAUTEUR_VERS_DUREE_IRRIGATION)

df_bilan.describe()

,etp,profondeur_enracinement,profondeur_terrefine,ru,rfu,rfu_deficit,precipitation,etm_culture,besoin_irrigation,rfu_cible,duree_irrigation
count,1096.000000,1096.0,1096.0,1.096000e+03,1.096000e+03,1096.0,1096.000000,1096.000000,1096.000000,1.096000e+03,1096.000000
mean,-2.261527,20.0,18.0,3.240000e+01,2.160000e+01,0.0,2.281077,-1.130763,-1.150314,2.160000e+01,5.885463
std,1.570648,0.0,0.0,7.108671e-15,3.554336e-15,0.0,4.186075,0.785324,4.418809,3.554336e-15,9.332213
min,-9.000000,20.0,18.0,3.240000e+01,2.160000e+01,0.0,0.000000,-4.500000,-37.742575,2.160000e+01,0.000000
25%,-3.321459,20.0,18.0,3.240000e+01,2.160000e+01,0.0,0.027787,-1.660730,-1.917946,2.160000e+01,0.000000
50%,-2.033154,20.0,18.0,3.240000e+01,2.160000e+01,0.0,0.292061,-1.016577,0.306438,2.160000e+01,0.000000
75%,-0.954882,20.0,18.0,3.240000e+01,2.160000e+01,0.0,2.765560,-0.477441,1.302919,2.160000e+01,13.029188
max,-0.000000,20.0,18.0,3.240000e+01,2.160000e+01,0.0,38.553198,-0.000000,4.500000,2.160000e+01,45.000000
